# Random Sequence Design

Design 60 6-base RBS sequences randomly:

A) Uniformly random.  
B) Make use of PWM frequency. 

Considering there would be some repeated sequences with the 61 Baseline designs and 60 bandit recommendations, we may need to design more than 60 RBS sequences (filter out).

In [1]:
import numpy as np
from itertools import product

In [2]:
# create all combos

combos = []
char_sets = ['A', 'G', 'C', 'T']
design_len = 6

for combo in product(char_sets, repeat= design_len):
    combo = ''.join(combo) 
    combos.append(combo)
    
assert len(combos) == len(char_sets) ** design_len

## Uniformly random

In [3]:
num_design_seq = 60 

randomA = np.random.choice(np.asarray(combos), num_design_seq, replace = False)
randomA

array(['TCACGC', 'GAGGAC', 'TTCAAA', 'GAAGCG', 'TACAGA', 'TGTATC',
       'GACATA', 'GAACCA', 'TCATAT', 'CCTGAT', 'TGTACC', 'ACTCCG',
       'TCTCAA', 'ACTCCC', 'ATATTT', 'TGTCTT', 'TTTCCT', 'TTGGAT',
       'GTAGTA', 'TATCGA', 'ACTGGA', 'TTCATA', 'GCGCCA', 'GAATGA',
       'GGAATT', 'TGATGT', 'AGGATA', 'TTGAGC', 'GTCGAA', 'TTAGAT',
       'GATCGT', 'CTCCAA', 'CGAAGT', 'TTCTTG', 'GTTTGT', 'GCCTGC',
       'GCGTTT', 'CAGTAT', 'CAGATT', 'GATGTA', 'GGTTAA', 'GCCTCG',
       'AACTCC', 'TCATAA', 'CTGGCG', 'CTTAGA', 'TACCCA', 'ACTGTA',
       'GTGACA', 'CACTCC', 'GCATGA', 'CATGTG', 'TTCGTC', 'GCGCCT',
       'AGGGGG', 'TGGAAA', 'CTCAGC', 'GATTTC', 'ACCCGT', 'GGAATC'],
      dtype='<U6')

## Random with PWM

wait for data.